### 데이터 불러오시고

In [5]:
raw_data = pd.read_csv("vehicles.csv")

### 쓸 컬럼들만 지정해주신뒤에 

In [6]:
start_df = raw_data[['id','price','year','odometer','drive','fuel','manufacturer','state','title_status','transmission','type','paint_color','cylinders','model','vin']]

### 결측치를 날리고, 'vin'이 중복되는 값들을 제거하면

In [12]:
start_df = start_df.dropna(axis=0)
df = start_df.loc[start_df['vin'].drop_duplicates(keep='last').index]
df.shape

(77339, 15)

### 77339개 row가 됩니다.

- 혹시 몰라서 head, tail 찍어봤습니다. 혹시 순서가 다르게 들어가거나 하실수 있으니 한번씩 확인해 주세요.
- 제가 찍어둔거 말고 새로 찍어보셔서 비교해보시는 쪽으로 활용해 보세요!

In [21]:
df.tail()

,id,price,year,odometer,drive,fuel,manufacturer,state,title_status,transmission,type,paint_color,cylinders,model,vin
509547,7035259793,33895,2016.0,49698.0,4wd,gas,ford,ut,clean,automatic,truck,blue,6 cylinders,f-150,1FTFW1EG9GFA33106
509548,7035257184,6500,2008.0,195000.0,rwd,gas,chevrolet,ut,clean,automatic,van,white,8 cylinders,express 3500,1GAHG39K981133844
509551,7035110469,7500,2014.0,99990.0,fwd,gas,nissan,ut,clean,manual,hatchback,white,4 cylinders,versa note,3N1CE2CP2EL410458
509555,7034929999,32000,2018.0,39457.0,4wd,gas,gmc,ut,clean,automatic,pickup,black,6 cylinders,canyon crew cab slt,1gtg6denxj1127455
509574,7034523643,24895,2019.0,6973.0,4wd,gas,jeep,ut,clean,automatic,SUV,white,4 cylinders,cherokee,1C4PJMLB0KD267506


In [22]:
df.head()

,id,price,year,odometer,drive,fuel,manufacturer,state,title_status,transmission,type,paint_color,cylinders,model,vin
0,7034441763,17899,2012.0,63500.0,4wd,gas,volkswagen,ut,clean,manual,hatchback,black,4 cylinders,golf r,WVWPF7AJ6CW316713
24,7034278551,4600,2008.0,110982.0,fwd,gas,honda,ut,clean,automatic,sedan,grey,4 cylinders,civic,2HGFA16528H503005
39,7033943607,3995,1999.0,215349.0,4wd,gas,subaru,ut,clean,automatic,wagon,blue,4 cylinders,outback,4S3BG6859X6660102
60,7033478714,28895,2016.0,19086.0,4wd,gas,ram,ut,clean,automatic,truck,red,8 cylinders,1500,1C6RR7GT6GS369389
61,7033472903,105500,2017.0,35000.0,4wd,diesel,mercedes-benz,ut,clean,other,van,blue,6 cylinders,sprinter 2500 4x4,WD4FE7CD1HP550108


### 4명이서 나누면 대략

In [20]:
77449 / 4

19362.25

### 요런 느낌으로 하면 될것 같습니다.
- cho : 0~20000
- hong : 20000~40000
- bae : 40000~60000
- yu : 60000~

### 함수를 만들어 봤습니다. 함수에 index 시작과 끝 숫자를 넣어주시면 됩니다.

In [27]:
def vin_cralwer(start_num,end_num):
    result_list = []
    result = {}
    for i in df.sort_values(by=['odometer'],ascending=False)[['vin','odometer','price']][start_num:end_num].iterrows():
        vin = i[1][0]
        odometer = i[1][1]
        url = 'https://marketvalue.vinaudit.com/getmarketvalue.php?key=1HB7ICF9L0GVH5Q&vin={}&mileage={}&country=USA'.format(vin, odometer)
        response = requests.get(url)
        jsonresponse = response.json()
        for k, v in jsonresponse.items():
            if k == 'prices':
                for key, value in v.items():
                    result[key] = value
            else:
                result[k] = v

        result_list.append(result)
        result = {}
    return pd.DataFrame(result_list).to_csv('./vin_crawling.csv')

- 작은 범위로 테스트 한번 해보시고 크롤링 해주시면 됩니다. 
- 위의 csv파일 저장되는 위치와 이름은 변경하셔도 됩니다.

In [28]:
vin_cralwer(0,20)